In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import pipeline
import torch

model_id = "MaziyarPanahi/Llama-3-8B-Instruct-v0.8"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2"
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)

streamer = TextStreamer(tokenizer)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    streamer=streamer
)

# Then you can use the pipeline to generate text.

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,    
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
)
print(outputs[0]["generated_text"][len(prompt):])


In [15]:
msg='''You are a men's fashion expert with extensive knowledge of clothing styles, clothing types and their combinations. I'm going to give you a clothing style, season for combinations, a list of clothing types with their parameters, and I want:

Combine types of clothing with each other if you think that the combination of clothes fits the specified style, season, and fits the look that is fashionable to wear. 
Provide all combinations. Exclude combinations, if they are not matching with the input conditions.


Input format:
Number of output outfits: 9

Clothing style: "{value}"
Season for outfits: "{value}"
List of clothing types: [
"{value1}",
"{value2}",
"{value3}",
…
]


output format:


List of outfits: [
{
"outerwear": "{value}",
"top": "{value}",
"bottom": "{value}",
"footwear": "{value}"
},
…
]

Categories top, bottom, footwear, outwear can not be null.

Now do the work for the following input:

Clothing style: "casual"
Season: "winter"
List of clothing types: [
"extended-length coat",
"regular-length coat",
"Slim-cut pullover",
"Oversize-cut pullover",
"Oversize-cut down-jacket",
"Regular-cut down-jacket",
"regular-length V-neck-neckline t-shirt", 
"Oversize-cut boat-neckline t-shirt", 
"regular-waist baggy-cut jeans", 
"high-waist baggy-cut jeans",
"regular-waist straight-cut jeans", 
"regular-waist flared-cut jeans", 
"low-top sneakers", 
"regular-length V-neck-neckline sweater",
"chukka_boots",
"uggs_boots",
"boots",
"dress_shoes"
]


If there are not enough items to match correspoing Number of output outfits then just finish
In output do not add additional information only output'''

In [ ]:
messages = [
    {"role": "user", "content": msg},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,    
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=10000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
)
print(outputs[0]["generated_text"][len(prompt):])


In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

# prepare data for llm

In [18]:
import pandas as pd
import json 
with open('data/female-style-type.json') as f:
    female_style_type = json.load(f)
with open('data/male-style-type.json') as f:
    male_style_type = json.load(f)

female_style_type_df=pd.DataFrame(female_style_type).explode('types').reset_index(drop=True)
male_style_type_df=pd.DataFrame(male_style_type).explode('types').reset_index(drop=True)

In [19]:
with open('data/female-types-with-parameters.json') as f:
    female_types_with_parameters = json.load(f)
with open('data/female-types.json') as f:
    female_types = json.load(f)
with open('data/male-types-with-parameters.json') as f:
    male_types_with_parameters = json.load(f)
with open('data/male-types.json') as f:
    male_types = json.load(f)


In [20]:
import itertools
def get_parameters_combinations(value):
    if type(value)!=list:
        return value
    result=[]
    for l in itertools.product(*value):
        result.append(list(l))
    return result

def get_param_value_union(list_values,column,column2):
    if type(list_values)!=list:
        return list_values
    interim=[]
    for i in list_values:
        params=i[column]
        params= [i[column2] + '-' + j for j in params]
        interim.append(params)
    return interim

def get_promt_text_params(clothing_type, parameters_combinations):
    if type(parameters_combinations)!=list:
        return clothing_type
    result=''
    for text in parameters_combinations:
        result+=' '+text
    # clothing_type=clothing_type.replace('_',' ')
    result=result+' '+clothing_type
    return result

In [21]:
def get_df_with_clothing_promt(gender_types,gender_types_with_parameters):
    gender_types_df=pd.DataFrame(gender_types)
    gender_types_with_parameters_df=pd.DataFrame(gender_types_with_parameters)
    df=gender_types_df.merge(gender_types_with_parameters_df, left_on='type', right_on='clothes_type',how='left')
            #flatten df to get more explicit view
    df['paramValueUnion']=df.parameters.apply(lambda x: get_param_value_union(x,'values','name'))
    df['paramCombinations']=df.paramValueUnion.apply(lambda x: get_parameters_combinations(x))
    df=df.explode('paramCombinations').reset_index(drop=True)
    df['promtTypeParams']=df.apply(lambda x: get_promt_text_params(clothing_type=x['type'],parameters_combinations=x['paramCombinations']),axis=1)
    return df 

In [22]:
female_df_clothing_promt=get_df_with_clothing_promt(gender_types=female_types,gender_types_with_parameters=female_types_with_parameters)
male_df_clothing_promt=get_df_with_clothing_promt(gender_types=male_types,gender_types_with_parameters=male_types_with_parameters)

In [27]:
male_df_clothing_promt[male_df_clothing_promt['category']=='top']['type'].unique()

array(['sweatshirt', 'hoodie', 'turtleneck', 'sweater', 'cardigan',
       'vest', 't-shirt', 'long_sleeve_t-shirt', 'polo',
       'short_sleeve_shirt', 'long_sleeve_shirt', 'denim_shirt',
       'linen_shirt', 'blazer'], dtype=object)

In [14]:
df_female_style_promt=female_style_type_df.merge(female_df_clothing_promt,left_on='types',right_on='type')
df_male_style_promt=male_style_type_df.merge(male_df_clothing_promt,left_on='types',right_on='type')

In [15]:
df_female_style_promt=df_female_style_promt.groupby(['styleType','category'],as_index=False)['promtTypeParams'].apply(list)
df_male_style_promt=df_male_style_promt.groupby(['styleType','category'],as_index=False)['promtTypeParams'].apply(list)

In [21]:
df_female_style_promt['promtTypeParams']=df_female_style_promt.apply(lambda x: {x['category']:x['promtTypeParams']},axis=1)
df_male_style_promt['promtTypeParams']=df_male_style_promt.apply(lambda x: {x['category']:x['promtTypeParams']},axis=1)


In [23]:
df_female_style_promt=df_female_style_promt.groupby(['styleType'],as_index=False)['promtTypeParams'].apply(list)
df_male_style_promt=df_male_style_promt.groupby(['styleType'],as_index=False)['promtTypeParams'].apply(list)

# llm part

16000 - комбинаций
9600 - женщины
6400 - мужчины
{'casual':3,'classic':1,'smart-casual':2,'street':1,old-money:1,oversized:1,minimalism:1}

In [1]:
import pandas as pd 
from ast import literal_eval
df_female_style_promt=pd.read_csv('data/female_style_promt.csv',index_col=0)
df_male_style_promt=pd.read_csv('data/male_style_promt.csv',index_col=0)
df_female_style_promt['promtTypeParams']=df_female_style_promt['promtTypeParams'].apply(literal_eval)
df_male_style_promt['promtTypeParams']=df_male_style_promt['promtTypeParams'].apply(literal_eval)


In [2]:
df_styleProportion=pd.DataFrame([{'casual':3,'classic':1,'smart-casual':2,'street':1,'old-money':1,'oversized':1,'minimalism':1}]).T.reset_index()\
    .rename(columns={'index':'styleType',0:'proportionPriority'})
df_styleProportion['proportionPriority']=df_styleProportion.proportionPriority/df_styleProportion.proportionPriority.sum()

In [3]:
df_female_style_promt=df_female_style_promt.merge(df_styleProportion,on='styleType')
df_male_style_promt=df_male_style_promt.merge(df_styleProportion,on='styleType')
df_female_style_promt['generationNumber']=(df_female_style_promt.proportionPriority*9600).astype(int)
df_male_style_promt['generationNumber']=(df_male_style_promt.proportionPriority*6400).astype(int)

In [4]:
df_female_style_promt['seasons']=[['winter','autumn','summer','spring']]*len(df_female_style_promt)
df_male_style_promt['seasons']=[['winter','autumn','summer','spring']]*len(df_male_style_promt)
df_female_style_promt['generationNumber']=df_female_style_promt.apply(lambda x: int(x['generationNumber']/len(x['seasons'])),axis=1)
df_male_style_promt['generationNumber']=df_male_style_promt.apply(lambda x: int(x['generationNumber']/len(x['seasons'])),axis=1)

In [5]:
df_female_style_promt=df_female_style_promt.explode('seasons').reset_index(drop=True)
df_male_style_promt=df_male_style_promt.explode('seasons').reset_index(drop=True)

In [6]:
list_promtTypeParams=df_male_style_promt['promtTypeParams'][0]
season='autumn'
style='casual'
number=5
tmplt_summer='''{"layered_top": "value",
"top": "value",
"bottom": "value",
"footwear": "value"}'''
tmplt_winter='''{"outerwear": "value",
"top": "value",
"bottom": "value",
"footwear": "value"}'''

In [ ]:
from src.msgs import get_right_msg
print(get_right_msg(style=style,
                  season=season,
                  number=number,
                  list_promtTypeParams=list_promtTypeParams,
                  tmplt=tmplt_winter))

In [8]:
from src.msgs import get_right_msg
from fireworks.client import Fireworks
def get_llm_response(msg):
    client = Fireworks(api_key="fw_3ZkkhpugiDMHrjoXdFZ6D44c")
    response = client.chat.completions.create(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    messages=[
    {
        "role": "system",
        "content": "You are fashion expert with extensive knowledge of clothing styles, clothing types and their combinations"
    },
    {
        "role": "user",
        "content": msg
    }
    ],)
    return response.choices[0].message.content


In [9]:
for i in range(3,len(df_male_style_promt)):
    gender='male'
    list_promtTypeParams=df_male_style_promt['promtTypeParams'][i]
    season=df_male_style_promt.seasons[i]
    style=df_male_style_promt.styleType[i]
    # number=df_male_style_promt.generationNumber[i]
    number=5
    if season=='summer' or season =='spring':
        tmplt='''{"layered_top": "value",
        "top": "value",
        "bottom": "value",
        "footwear": "value"}'''
    else:
        tmplt='''{"outerwear": "value",
        "top": "value",
        "bottom": "value",
        "footwear": "value"}'''

    msg=get_right_msg(style=style,
                  season=season,
                  number=number,
                  list_promtTypeParams=list_promtTypeParams,
                  tmplt=tmplt)
    output=get_llm_response(msg)
    with open(f"'data_llm/outputs/'{season}_{style}_{gender}_{number}.txt", "w") as text_file:
        text_file.write(output)

FileNotFoundError: [Errno 2] No such file or directory: "'data_llm/outputs/'spring_casual_male_5.txt"

In [13]:
with open(f"data_llm/outputs/{season}_{style}_{gender}_{number}.txt", "w") as text_file:
    text_file.write(output)

In [11]:
literal_eval(output.replace('null',"'empty'"))

[{'layered_top': 'empty',
  'top': 'fit-regular sweatshirt',
  'bottom': 'fit-tapered waist-regular length-cropped joggers',
  'footwear': 'low-top_sneakers'},
 {'layered_top': 'empty',
  'top': 'fit-slim neckline-round_neck sweater',
  'bottom': 'waist-regular fit-tapered length-cropped casual_trousers',
  'footwear': 'high-top_sneakers'},
 {'layered_top': 'leather_jacket',
  'top': 'fit-regular neckline-round_neck t-shirt',
  'bottom': 'waist-regular fit-straight length-cropped jeans',
  'footwear': 'boots'},
 {'layered_top': 'empty',
  'top': 'fit-oversize hoodie',
  'bottom': 'fit-wide waist-regular length-cropped joggers',
  'footwear': 'low-top_sneakers'},
 {'layered_top': 'denim_jacket',
  'top': 'fit-slim neckline-V-neck t-shirt',
  'bottom': 'waist-regular fit-tapered length-cropped linen_trousers',
  'footwear': 'chelsea_boots'}]